In [ ]:
import pandas as pd
import json
import csv
import matplotlib.pyplot as plt
import string
import unicodedata as ud
import nltk
from collections import Counter
from nltk import ngrams


In [ ]:
!pip install hazm
!pip install python-docx 

from docx import Document
from hazm import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unrar x "/content/drive/MyDrive/ferdowsi-data.rar"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/MyDrive/ferdowsi-data.rar

Creating    Spad                                                      OK
Extracting  Spad/ferdowsi-data.txt                                         0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 62% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75% 76% 77% 78% 79% 80% 81% 82% 83% 84% 85% 86% 87% 88% 89% 90

In [ ]:
file = open('/content/Spad/ferdowsi-data.txt', 'r')
df = pd.DataFrame()

#persian_punc = frozenset(string.punctuation + "،؟")
#bool(set("test?") & test_characters)




NUM_ENTRIES = 1000 #number of entries to fetch 


In [ ]:
cnt = 0
cur_entry = ''

for _ in range(NUM_ENTRIES*16):
    cnt += 1
    line = file.readline()
    cur_entry += line
    try:
        if cnt % 16 == 0:
            row = json.loads(cur_entry)
            row = row['_source']
           #print(row)
            df = df.append(row, ignore_index=True)
            cur_entry = ''
    except:
        pass

file.close()


In [ ]:
df

,agency,category,content,date_created,image,lead,title,url
0,13,15,فهیمه حسن‌میری: اگرچه در هیاهوی انتخابات ریاس...,1.494518e+09,uploads/2017/05/11/1589793661.jpg,جامعه&nbsp;&gt;&nbsp;شهری - میزگردی درباره عمل...,کارهای قالیباف در تهران نمایشی بود- شورای‌شهر ...,http://www.khabaronline.ir/(X(1)S(bud4wg3ebzbx...
1,1,20,به گزارش خبرگزاری فارس از قم، محمدعلی برزنونی ...,1.494522e+09,uploads/2017/05/11/1713799235.jpg,مدیر مؤسسه علمی و پژوهشی ابن‌سینا در بوسنی گفت...,معنویت در عصر آخرالزمان احیا می‌شود,http://www.farsnews.com/13960221001386
2,36,5,به گزارش ایرنا، تیم های تراکتورسازی تبریز و...,1.494522e+09,uploads/2017/05/11/2561366787.jpg,تهران - ایرنا - دیدار پایانی جام حذفی باشگاه ه...,فینال جام حذفی؛ وداع البرز حاجی پور از داوری-3...,http://www.irna.ir/fa/News/82525960/
3,13,8,به گزارش خبرگزاری خبرآنلاین، جمشید مشایخی، هن...,1.494522e+09,uploads/2017/05/11/109987278.jpg,فرهنگ&nbsp;&gt;&nbsp;سینما - جمشید مشایخی، امر...,موضع انتخاباتی جمشید مشایخی روی تخت بیمارستان-...,http://www.khabaronline.ir/(X(1)S(nducmtisxqhw...
4,13,7,به گزارش خبرآنلاین، پیش‌تر گفته شد سامسونگ و ...,1.494522e+09,uploads/2017/05/11/3295317773.jpg,فناوری اطلاعات&nbsp;&gt;&nbsp;سخت افزار - اسنپ...,مشخصات فنی پردازنده اسنپ دراگون ۸۴۵- رونمایی ا...,http://www.khabaronline.ir/(X(1)S(uupfbn0kvlth...
...,...,...,...,...,...,...,...,...
995,2,15,سردار بخشعلی کامرانی صالح روز شنبه درگفت و گو...,1.494660e+09,uploads/2017/05/13/4034459417.jpg,کرج -ایرنا- فرمانده انتظامی استان البرز گفت که...,مبارزه با کالای قاچاق به عزم ملی و مشارکت گستر...,http://www.irna.ir/fa/News/82527216/
996,33,31,به گزارش خبرگزاری صداوسیما، مرکز خراسان رضوی...,1.494660e+09,False,سفیر استرالیا در ایران گفت: شاهنامه فردوسی از ...,شاهنامه فردوسی نماینده یکی از بزرگترین تمدن ها...,http://www.iribnews.ir/fa/news/1630788/%D8%B4%...
997,36,5,نسیمی شاد در رشته 50 متر قورباغه با ثبت زما...,1.494661e+09,uploads/2017/05/13/2941477288.jpg,باکو - ایرنا - «آریا نسیمی شاد» شناگر ایرانی د...,بازی های کشورهای اسلامی- نمایندگان شنا، کاراته...,http://www.irna.ir/fa/News/82527314/
998,33,1,به گزارش خبرنگار خبرگزاری صدا و سیما، نمایندگ...,1.494661e+09,False,جلسه هماهنگی شورای نگهبان با نمایندگان نامزدها...,آغاز جلسه هماهنگی شورای نگهبان با نمایندگان نا...,http://www.iribnews.ir/fa/news/1630875/%D8%A2%...


In [ ]:
stemmer = Stemmer()
normalizer = Normalizer()
lemmatizer = Lemmatizer()

**Part 1 -- Normalizing & finding most frequent words & bigrams (including stop words)**

In [ ]:
df['content'] = df['content'].apply(lambda x: normalizer.normalize(x)) #making sure half spaces correct

In [ ]:
def lemmatize_sentence(sentence):
  s = ''
  for word in word_tokenize(sentence):
    #s += stemmer.stem(word)+ " "
    try: 
      s +=  lemmatizer.lemmatize(word).split("#")[1]+ " "
    except: 
      s +=  lemmatizer.lemmatize(word).split("#")[0]+ " "
  return s


In [ ]:
'''before we lemmatize each of the sentences, lets remove their puncuation'''

punc_table = str.maketrans(dict.fromkeys(string.punctuation + "؟،«»؛"))
df['content_normalized'] = df['content'].apply(lambda x: x.translate(punc_table))



'''Now for the lemmatizing!'''

df['content_normalized'] = df['content_normalized'].apply(lambda x: lemmatize_sentence(x))

In [ ]:
tokenized_words_list = word_tokenize(" ".join(df['content_normalized'])) #the " ".join part is to get all into one giant string

Finding most common words and bigrams (without removing stopwords):

In [ ]:
Counter(tokenized_words_list).most_common(10)

[('و', 20174),
 ('در', 15879),
 ('به', 13579),
 ('از', 9616),
 ('که', 9235),
 ('این', 8472),
 ('کن', 7880),
 ('را', 7522),
 ('با', 6412),
 ('شو', 5835)]

In [ ]:
bigram_counts = Counter(ngrams(tokenized_words_list, 2))
bigram_counts.most_common(10)

[(('است', 'که'), 1368),
 (('در', 'این'), 1302),
 (('کن', 'و'), 1057),
 (('که', 'در'), 1029),
 (('را', 'به'), 931),
 (('به', 'گزارش'), 789),
 (('شو', 'و'), 727),
 (('و', 'در'), 665),
 (('را', 'در'), 650),
 (('کن', 'که'), 636)]

**Part 2**

persian stopwords source: [link1](https://bigdata-ir.com/ایست‌واژه‌ها-یا-stop-word-در-زبان-فارسی/) (stopwords-Farsi1) and [link2](https://github.com/kharazi/persian-stopwords/blob/master/persian) (stopwords-Farsi2)

In [ ]:
document = Document('/content/drive/MyDrive/datasets/stopwords-Farsi2.docx')

persian_stopwords = []

for para in document.paragraphs:
    persian_stopwords.append(para.text)

In [ ]:
def remove_stopwords(sentence):
  querywords = sentence.split()
  resultwords  = [word for word in querywords if word not in persian_stopwords]
  return ' '.join(resultwords)

In [ ]:
'''lets remove their puncuation'''

punc_table = str.maketrans(dict.fromkeys(string.punctuation + "؟،«»؛"))
df['content_normalized'] = df['content'].apply(lambda x: x.translate(punc_table))


'''removing the stopwords...'''

df['content_normalized'] = df['content_normalized'].apply(lambda x: remove_stopwords(x))


'''Now for the lemmatizing!'''

df['content_normalized'] = df['content_normalized'].apply(lambda x: lemmatize_sentence(x))

In [ ]:
tokenized_words_list = word_tokenize(" ".join(df['content_normalized'])) #the " ".join part is to get all into one giant string

In [ ]:
txt = df['content_normalized']
Counter(tokenized_words_list).most_common(10)

[('کشور', 2157),
 ('سال', 1847),
 ('دولت', 1804),
 ('ایران', 1389),
 ('کن', 1360),
 ('کار', 1287),
 ('اقتصاد', 1244),
 ('شهر', 1077),
 ('انتخابات', 1066),
 ('گزارش', 996)]

In [ ]:
bigram_counts = Counter(ngrams(tokenized_words_list, 2))
bigram_counts.most_common(10)

[(('ریاست', 'جمهوری'), 437),
 (('انتخابات', 'ریاست'), 288),
 (('شورا', 'شهر'), 265),
 (('دولت', 'یازدهم'), 222),
 (('انتهای', 'پیام'), 185),
 (('رئیس', 'جمهور'), 170),
 (('گزارش', 'خبرنگار'), 162),
 (('فرزند', 'محمد'), 160),
 (('شورا', 'اسلام'), 155),
 (('اقتصاد', 'مقاومت'), 146)]

seems like we're getting good results :)

**Part 3**

In [ ]:
df['num_sentences'] = df['content'].apply(lambda x: len(sent_tokenize(x)))

In [ ]:
df

,agency,category,content,date_created,image,lead,title,url,content_normalized,num_sentences
0,13,15,فهیمه حسن‌میری: اگرچه در هیاهوی انتخابات ریاس...,1.494518e+09,uploads/2017/05/11/1589793661.jpg,جامعه&nbsp;&gt;&nbsp;شهری - میزگردی درباره عمل...,کارهای قالیباف در تهران نمایشی بود- شورای‌شهر ...,http://www.khabaronline.ir/(X(1)S(bud4wg3ebzbx...,فهیمه حسن‌میری هیاهو انتخابات ریاست جمهوری موض...,150
1,1,20,به گزارش خبرگزاری فارس از قم، محمدعلی برزنونی ...,1.494522e+09,uploads/2017/05/11/1713799235.jpg,مدیر مؤسسه علمی و پژوهشی ابن‌سینا در بوسنی گفت...,معنویت در عصر آخرالزمان احیا می‌شود,http://www.farsnews.com/13960221001386,گزارش خبرگزاری فارس قم محمدعلی برزنونی عصر کمی...,11
2,36,5,به گزارش ایرنا، تیم‌های تراکتورسازی تبریز و ن...,1.494522e+09,uploads/2017/05/11/2561366787.jpg,تهران - ایرنا - دیدار پایانی جام حذفی باشگاه ه...,فینال جام حذفی؛ وداع البرز حاجی پور از داوری-3...,http://www.irna.ir/fa/News/82525960/,گزارش ایرنا تیم‌های تراکتورسازی تبریز نفت تهرا...,13
3,13,8,به گزارش خبرگزاری خبرآنلاین، جمشید مشایخی، هن...,1.494522e+09,uploads/2017/05/11/109987278.jpg,فرهنگ&nbsp;&gt;&nbsp;سینما - جمشید مشایخی، امر...,موضع انتخاباتی جمشید مشایخی روی تخت بیمارستان-...,http://www.khabaronline.ir/(X(1)S(nducmtisxqhw...,گزارش خبرگزاری خبرآنلاین جمشید مشایخ هنرمند پی...,15
4,13,7,به گزارش خبرآنلاین، پیش‌تر گفته شد سامسونگ و ...,1.494522e+09,uploads/2017/05/11/3295317773.jpg,فناوری اطلاعات&nbsp;&gt;&nbsp;سخت افزار - اسنپ...,مشخصات فنی پردازنده اسنپ دراگون ۸۴۵- رونمایی ا...,http://www.khabaronline.ir/(X(1)S(uupfbn0kvlth...,گزارش خبرآنلاین پیش سامسونگ TSMC تایوان قصد سا...,8
...,...,...,...,...,...,...,...,...,...,...
995,2,15,سردار بخشعلی کامرانی صالح روز شنبه درگفت و گو...,1.494660e+09,uploads/2017/05/13/4034459417.jpg,کرج -ایرنا- فرمانده انتظامی استان البرز گفت که...,مبارزه با کالای قاچاق به عزم ملی و مشارکت گستر...,http://www.irna.ir/fa/News/82527216/,سردار بخشعلی کامران صالح شنبه درگفت خبرنگار ای...,5
996,33,31,به گزارش خبرگزاری صداوسیما، مرکز خراسان رضوی،...,1.494660e+09,False,سفیر استرالیا در ایران گفت: شاهنامه فردوسی از ...,شاهنامه فردوسی نماینده یکی از بزرگترین تمدن ها...,http://www.iribnews.ir/fa/news/1630788/%D8%B4%...,گزارش خبرگزاری صداوسیما مرکز خراسان رضوی ایان ...,4
997,36,5,نسیمی شاد در رشته ۵۰ متر قورباغه با ثبت زمان ...,1.494661e+09,uploads/2017/05/13/2941477288.jpg,باکو - ایرنا - «آریا نسیمی شاد» شناگر ایرانی د...,بازی های کشورهای اسلامی- نمایندگان شنا، کاراته...,http://www.irna.ir/fa/News/82527314/,نسیم شاد ۵۰ متر قورباغه ثبت ۲۹ ثانیه ۵۸ صدم ثا...,12
998,33,1,به گزارش خبرنگار خبرگزاری صدا و سیما، نمایندگ...,1.494661e+09,False,جلسه هماهنگی شورای نگهبان با نمایندگان نامزدها...,آغاز جلسه هماهنگی شورای نگهبان با نمایندگان نا...,http://www.iribnews.ir/fa/news/1630875/%D8%A2%...,گزارش خبرنگار خبرگزاری صدا سیما نمایندگان جلسه...,2
